# HLCA Datasets gene harmonisation

This notebook is in charge of using idtrack for the gene harmonisation of the HLCA.

__author__ = "Kemal Inecik, Ciro Ramírez-Suástegui"

__maintainer__ = "Lisa Sikkema, Ciro Ramírez-Suástegui"

__email__ = "lisa.sikkema@helmholtz-muenchen.de, ciro.rsuastegui@helmholtz-munich.de"

### Import

In [3]:
import os
import time
import pickle

In [ ]:
import scanpy as sc
import idtrack

In [3]:
%load_ext autoreload
%autoreload 2

### Load data

Initialize the graph and pathfinder

In [4]:
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"
idt = idtrack.API(local_repository=local_dir)
idt.configure_logger()
idt.initialize_graph(organism_name='homo_sapiens', ensembl_release=107, return_test=True)

2022-11-01 16:45:08 INFO:graph_maker: The graph is being read.


In [5]:
idt.calculate_graph_caches() 

2022-11-01 16:45:47 INFO:the_graph: Cached properties being calculated: combined_edges
2022-11-01 16:46:32 INFO:the_graph: Cached properties being calculated: combined_edges_assembly_specific_genes
2022-11-01 16:46:35 INFO:the_graph: Cached properties being calculated: combined_edges_genes
2022-11-01 16:46:46 INFO:the_graph: Cached properties being calculated: lower_chars_graph
2022-11-01 16:46:47 INFO:the_graph: Cached properties being calculated: get_active_ranges_of_id
2022-11-01 16:47:06 INFO:the_graph: Cached properties being calculated: available_external_databases
2022-11-01 16:47:07 INFO:the_graph: Cached properties being calculated: external_database_connection_form
2022-11-01 16:47:40 INFO:the_graph: Cached properties being calculated: available_genome_assemblies
2022-11-01 16:47:40 INFO:the_graph: Cached properties being calculated: available_external_databases_assembly
2022-11-01 16:47:41 INFO:the_graph: Cached properties being calculated: node_trios


The datasets of HLCA

In [6]:
base_path = "/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data"
dset0_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/ready/full")
dset1_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/raw")

adata_dict = {
    "Kaminski_2020": [f"{dset0_dir}/adams.h5ad"],
    "Meyer_2021": [f"{dset0_dir}/meyer_2021.h5ad"],
    "MeyerNikolic_unpubl": [f"{dset0_dir}/meyer_nikolic_unpubl.h5ad"],
    "Barbry_unpubl": [f"{dset0_dir}/barbry.h5ad"],
    "Regev_2021": [
        f"{dset0_dir}/delorey_cryo.h5ad", f"{dset0_dir}/delorey_fresh.h5ad",
        f"{dset0_dir}/delorey_nuclei.h5ad"
    ],
    "Thienpont_2018": [f"{dset1_dir}/Lambrechts/lambrechts.h5ad"],
    "Budinger_2020": [f"{dset0_dir}/bharat.h5ad"],
    "Banovich_Kropski_2020": [f"{dset0_dir}/haberman.h5ad"],
    "Sheppard_2020": [f"{dset0_dir}/tsukui.h5ad"],
    "Wunderink_2021": [
        f"{dset0_dir}/grant_cryo.h5ad", f"{dset0_dir}/grant_fresh.h5ad"
    ],
    "Lambrechts_2021": [
        f"{dset0_dir}/wouters.h5ad" #, f"{dset0_dir}/wouters_labs.h5ad"
    ],
    "Zhang_2021": [f"{dset1_dir}/Liao/covid_for_publish.h5ad"],
    "Duong_lungMAP_unpubl": [f"{dset0_dir}/duong.h5ad"],
    "Janssen_2020": [f"{dset0_dir}/mould.h5ad"],
    "Sun_2020": [
        f"{dset0_dir}/wang_sub_batch1.h5ad", f"{dset0_dir}/wang_sub_batch2.h5ad",
        f"{dset0_dir}/wang_sub_batch3.h5ad", f"{dset0_dir}/wang_sub_batch4.h5ad"],
    "Gomperts_2021": [
        f"{dset0_dir}/carraro_ucla.h5ad", f"{dset0_dir}/carraro_cff.h5ad",
        f"{dset0_dir}/carraro_csmc.h5ad"],
    "Eils_2020": [f"{dset0_dir}/lukassen.h5ad"],
    "Schiller_2020": [f"{dset0_dir}/mayr.h5ad"],
    "Misharin_Budinger_2018": [f"{dset0_dir}/reyfman_disease.h5ad"],
    "Shalek_2018": [f"{dset0_dir}/ordovasmontanes.h5ad"],
    "Schiller_2021": [f"{dset0_dir}/schiller_discovair.h5ad"],
    "Peer_Massague_2020": [f"{dset0_dir}/laughney.h5ad"],
    "Lafyatis_2019": [f"{dset0_dir}/valenzi.h5ad"],
    "Tata_unpubl": [f"{dset0_dir}/tata_unpubl.h5ad"],
    "Xu_2020": [f"{dset0_dir}/guo.h5ad"],
    "Sims_2019": [f"{dset0_dir}/szabo.h5ad"],
    "Schultze_unpubl": [f"{dset0_dir}/schultze_unpubl.h5ad"]
}

Run the ID conversion with HGNC Symbol (a.k.a gene name)

In [7]:
result = dict()
final_database="HGNC Symbol"

for dataset_name in adata_dict:
    
    adata = sc.read(adata_dict[dataset_name][0])
    gene_list = list(adata.var.index)
    
    matching = idt.convert_identifier_multiple(gene_list, final_database=final_database, pbar_prefix=dataset_name)
    binned_conversions = idt.classify_multiple_conversion(matching)
    
    idt.print_binned_conversion(binned_conversions)
    print(f"Source release: {idt.infer_identifier_source(gene_list)}")
    
    result[dataset_name] = binned_conversions

Kaminski_2020:   0%|          | 0/45947 [00:00<?, ?it/s]

changed_only_1_to_n: 59
changed_only_1_to_1: 4002
alternative_target_1_to_1: 12331
alternative_target_1_to_n: 13
matching_1_to_0: 98
matching_1_to_1: 33446
matching_1_to_n: 59
input_identifiers: 45947
Source release: (38, 94)


Meyer_2021:   0%|          | 0/20922 [00:00<?, ?it/s]

changed_only_1_to_n: 3
changed_only_1_to_1: 997
alternative_target_1_to_1: 368
alternative_target_1_to_n: 2
matching_1_to_0: 17
matching_1_to_1: 20532
matching_1_to_n: 3
input_identifiers: 20922
Source release: (38, 84)


MeyerNikolic_unpubl:   0%|          | 0/33582 [00:00<?, ?it/s]

...

changed_only_1_to_n: 985
changed_only_1_to_1: 8062
alternative_target_1_to_1: 16431
alternative_target_1_to_n: 265
matching_1_to_0: 1461
matching_1_to_1: 41583
matching_1_to_n: 985
input_identifiers: 60725


2022-11-01 18:45:32 WARNING:api: Number of unfound IDs: 10.


Source release: (38, 83)


Schultze_unpubl:   0%|          | 0/24532 [00:00<?, ?it/s]

changed_only_1_to_n: 5
changed_only_1_to_1: 2070
alternative_target_1_to_1: 3820
alternative_target_1_to_n: 6
matching_1_to_0: 46
matching_1_to_1: 20655
matching_1_to_n: 5
input_identifiers: 24532
Source release: (38, 91)


In [8]:
time_suffix = time.strftime("%Y%m%d-%H%M%S", time.gmtime())
file_path = os.path.join(local_dir, f"results_for_hlca_datasets_{final_database}_{time_suffix}.pk")
with open(file_path, 'wb') as handle:
    pickle.dump(result, handle)
    print(f"Saved: {file_path}")

Saved: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/results_for_hlca_datasets_HGNC Symbol_20221101-175029.pk


Run the ID conversion with Ensembl gene ID

In [9]:
result = dict()
final_database="ensembl_gene"

for dataset_name in adata_dict:
    
    adata = sc.read(adata_dict[dataset_name][0])
    gene_list = list(adata.var.index)
    
    matching = idt.convert_identifier_multiple(gene_list, final_database=final_database, pbar_prefix=dataset_name)
    binned_conversions = idt.classify_multiple_conversion(matching)
    
    idt.print_binned_conversion(binned_conversions)
    print(f"Source release: {idt.infer_identifier_source(gene_list)}")
    
    result[dataset_name] = binned_conversions

Kaminski_2020:   0%|          | 0/45947 [00:00<?, ?it/s]

changed_only_1_to_n: 498
changed_only_1_to_1: 45351
alternative_target_1_to_1: 0
alternative_target_1_to_n: 0
matching_1_to_0: 98
matching_1_to_1: 45351
matching_1_to_n: 498
input_identifiers: 45947
Source release: (38, 94)


Meyer_2021:   0%|          | 0/20922 [00:00<?, ?it/s]

changed_only_1_to_n: 213
changed_only_1_to_1: 20692
alternative_target_1_to_1: 0
alternative_target_1_to_n: 0
matching_1_to_0: 17
matching_1_to_1: 20692
matching_1_to_n: 213
input_identifiers: 20922
Source release: (38, 84)


MeyerNikolic_unpubl:   0%|          | 0/33582 [00:00<?, ?it/s]

...

changed_only_1_to_n: 1830
changed_only_1_to_1: 57434
alternative_target_1_to_1: 0
alternative_target_1_to_n: 0
matching_1_to_0: 1461
matching_1_to_1: 57434
matching_1_to_n: 1830
input_identifiers: 60725


2022-11-01 20:09:57 WARNING:api: Number of unfound IDs: 10.


Source release: (38, 83)


Schultze_unpubl:   0%|          | 0/24532 [00:00<?, ?it/s]

changed_only_1_to_n: 196
changed_only_1_to_1: 24290
alternative_target_1_to_1: 0
alternative_target_1_to_n: 0
matching_1_to_0: 46
matching_1_to_1: 24290
matching_1_to_n: 196
input_identifiers: 24532
Source release: (38, 91)


### Save

In [10]:
time_suffix = time.strftime("%Y%m%d-%H%M%S", time.gmtime())
file_path = os.path.join(local_dir, f"results_for_hlca_datasets_{final_database}_{time_suffix}.pk")
with open(file_path, 'wb') as handle:
    pickle.dump(result, handle)
    print(f"Saved: {file_path}")

Saved: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/results_for_hlca_datasets_ensembl_gene_20221101-191345.pk
